# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

# Challenge 1 - Accessing our database
Create a connection to access the sakila database with the ip, user and password provided in class. Take a look at the data. Do some joins in order to have more elaborated tables in a dataframe (you can try first the query in the DBMS and then use it in your notebook).


In [2]:
# your code here
#First, import Pandas
import pandas as pd

In [3]:
#Second, import create engine from sqlalchemy
from sqlalchemy import create_engine

In [5]:
#Third, make the connection
db_path = '/home/alvaro/Downloads/juliarochflores.db'

conn_str = f'sqlite:///{db_path}'

engine = create_engine(conn_str)

In [7]:
#Now that we have made the connection, we can see what's inside, for example, the name of the Tables

pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)

,name
0,rank_info
1,personal_info
2,business_info


In [9]:
#Or what's inside each table
rank_info = pd.read_sql_query("select * from rank_info", engine)

rank_info.head()

,id,name,position,Unnamed: 0
0,6367,ananT ASAVABHOKHIN & family,NaN,0
1,7613,arash FERDOWSI,NaN,1
2,6139,banthoon LAMSAM & FAMILY,NaN,2
3,5671,bill ADDERLEY & family,NaN,3
4,2117,boris zINGArevich,NaN,4


In [12]:
#We can even merge all the tables to see it's content in just one table

merge_tables = """Select * from rank_info
join personal_info on personal_info.id = rank_info.id
join business_info on business_info.id = rank_info.id"""

merged_info = pd.read_sql_query(merge_tables, engine)
merged_info.head()

,id,name,position,Unnamed: 0,id,lastName,age,Unnamed: 0,gender,country,image,id,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
0,8760,jefF BEZOs,1.0,52,8760,BEZOS,54 years old,52,None,United States,https://specials-images.forbesimg.com/imageser...,8760,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,1
1,7500,bILL GATES,2.0,53,7500,GATES,62 years old,53,Male,None,https://specials-images.forbesimg.com/imageser...,7500,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,2
2,5751,WARREN BUFFEtt,3.0,54,5751,bufFETT,87 years old,54,M,None,https://specials-images.forbesimg.com/imageser...,5751,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,None,3
3,8365,bernARD ARNAULT,4.0,55,8365,arNAULt,69 years old,55,None,France,https://specials-images.forbesimg.com/imageser...,8365,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,None,4
4,5328,maRK ZUCKERBERG,5.0,56,5328,zUCKERBERg,1985,56,None,United States,https://specials-images.forbesimg.com/imageser...,5328,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,None,5


# Challenge 2 - Working with JSON files

Import the pandas library.

In [13]:
# your code here
import pandas as pd

In the next cell, load the data in `nasa.json` in the `data` folder and load it into a pandas dataframe. Name the dataframe `nasa`.

In [16]:
# your code here

nasa = pd.read_json('/home/alvaro/Bootcamp/dataptmad1120/module-1/importing-exporting-data/data/nasa.json', orient='records')

Now that we have loaded the data, let's examine it using the `head()` function.

In [15]:
# your code here
nasa.head()

,:@computed_region_cbhk_fwbd,:@computed_region_nnqa_25f4,fall,geolocation,id,mass,name,nametype,recclass,reclat,reclong,year
0,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [6.08333, 50....",1,21.0,Aachen,Valid,L5,50.77500,6.08333,1880-01-01T00:00:00.000
1,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [10.23333, 56...",2,720.0,Aarhus,Valid,H6,56.18333,10.23333,1951-01-01T00:00:00.000
2,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-113, 54.216...",6,107000.0,Abee,Valid,EH4,54.21667,-113.00000,1952-01-01T00:00:00.000
3,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-99.9, 16.88...",10,1914.0,Acapulco,Valid,Acapulcoite,16.88333,-99.90000,1976-01-01T00:00:00.000
4,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-64.95, -33....",370,780.0,Achiras,Valid,L6,-33.16667,-64.95000,1902-01-01T00:00:00.000


#### The `value_counts()` function is commonly used in pandas to find the frequency of every value in a column.

In the cell below, use the `value_counts()` function to determine the frequency of all types of asteroid landings by applying the function to the `fall` column.

In [18]:
# your code here

nasa['fall'].value_counts()

Fell     996
Found      4
Name: fall, dtype: int64

Finally, let's save the dataframe as a json file again. Save the dataframe using the `orient=records` argument and name the file `nasa-output.json`. Remember to save the file inside the `data` folder.

In [19]:
# your code here
nasa.to_json('/home/alvaro/Bootcamp/dataptmad1120/module-1/importing-exporting-data/data/nasa-output.json', orient='records')

# Challenge 2 - Working with CSV and Other Separated Files

CSV files are more commonly used as dataframes. In the cell below, load the file from the URL provided using the `read_csv()` function in pandas. Starting version 0.19 of pandas, you can load a CSV file into a dataframe directly from a URL without having to load the file first and then transform it. The dataset we will be using contains information about NASA shuttles.

In the cell below, we define the column names and the URL of the data. Following this cell, read the tst file to a variable called `shuttle`. Since the file does not contain the column names, you must add them yourself using the column names declared in `cols` using the `names` argument. Additionally, a tst file is space separated, make sure you pass ` sep=' '` to the function.

In [1]:
cols = ['time', 'rad_flow', 'fpv_close', 'fpv_open', 'high', 'bypass', 'bpv_close', 'bpv_open', 'class']
tst_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/shuttle/shuttle.tst'

In [8]:
# your code here

Let's verify that this worked by looking at the `head()` function.

In [9]:
# your code here

To make life easier for us, let's turn this dataframe into a comma separated file by saving it using the `to_csv()` function. Save `shuttle` into the file `shuttle.csv` and ensure the file is comma separated, that we are not saving the index column and that the file is saved inside the `data` folder.

In [10]:
# your code here

# Challenge 3 - Working with Excel Files

We can also use pandas to convert excel spreadsheets to dataframes. Let's use the `read_excel()` function. In this case, `astronauts.xls` is in the `data` folder. Read this file into a variable called `astronaut`. 

Note: Make sure to install the `xlrd` library if it is not yet installed.

In [11]:
# your code here

Use the `head()` function to inspect the dataframe.

In [12]:
# your code here

Use the `value_counts()` function to find the most popular undergraduate major among all astronauts.

In [13]:
# your code here

Due to all the commas present in the cells of this file, let's save it as a tab separated csv file. In the cell below, save `astronaut` as a **tab separated file** using the `to_csv` function. Call the file `astronaut.csv`. Remember to remove the index column and save the file in the `data` folder.

In [14]:
# your code here

# Bonus Challenge - Fertility Dataset

Visit the following [URL](https://archive.ics.uci.edu/ml/datasets/Fertility) and retrieve the dataset as well as the column headers. Determine the correct separator and read the file into a variable called `fertility`. Examine the dataframe using the `head()` function. 

Look in Google for a way to retrieve this data!

In [15]:
# your code here